In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.feature import StandardScaler
import pandas as pd

In [2]:
if __name__ == "__main__":
    sc = SparkSession .builder\
        .appName("standardScalar")\
        .getOrCreate()

In [3]:
df = sc.read.csv("diabetes.csv",header=True)

In [4]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [5]:
#we do not need to scale our label coulmn, so let's seperate it
features = df.drop('Outcome')


In [6]:
label = df.select('Outcome')

In [7]:
label.show()

+-------+
|Outcome|
+-------+
|      1|
|      0|
|      1|
|      0|
|      1|
|      0|
|      1|
|      0|
|      1|
|      1|
|      0|
|      1|
|      0|
|      1|
|      1|
|      1|
|      1|
|      1|
|      0|
|      1|
+-------+
only showing top 20 rows



In [8]:
features.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|
|         10|    115|            0|            0|      0|35.3|                  

In [9]:
col_names = features.columns
features_rdd = features.rdd.map(lambda row: row[0:])

In [10]:
features_rdd.collect()

[('6', '148', '72', '35', '0', '33.6', '0.627', '50'),
 ('1', '85', '66', '29', '0', '26.6', '0.351', '31'),
 ('8', '183', '64', '0', '0', '23.3', '0.672', '32'),
 ('1', '89', '66', '23', '94', '28.1', '0.167', '21'),
 ('0', '137', '40', '35', '168', '43.1', '2.288', '33'),
 ('5', '116', '74', '0', '0', '25.6', '0.201', '30'),
 ('3', '78', '50', '32', '88', '31', '0.248', '26'),
 ('10', '115', '0', '0', '0', '35.3', '0.134', '29'),
 ('2', '197', '70', '45', '543', '30.5', '0.158', '53'),
 ('8', '125', '96', '0', '0', '0', '0.232', '54'),
 ('4', '110', '92', '0', '0', '37.6', '0.191', '30'),
 ('10', '168', '74', '0', '0', '38', '0.537', '34'),
 ('10', '139', '80', '0', '0', '27.1', '1.441', '57'),
 ('1', '189', '60', '23', '846', '30.1', '0.398', '59'),
 ('5', '166', '72', '19', '175', '25.8', '0.587', '51'),
 ('7', '100', '0', '0', '0', '30', '0.484', '32'),
 ('0', '118', '84', '47', '230', '45.8', '0.551', '31'),
 ('7', '107', '74', '0', '0', '29.6', '0.254', '31'),
 ('1', '103', '30'

In [11]:
scaler1 = StandardScaler().fit(features_rdd)
#scaler2 = StandardScaler(withMean=True, withStd=True).fit(features)

In [12]:
scaled_features=scaler1.transform(features_rdd)

In [13]:
for data in scaled_features.collect():
    print(data)


[1.7806383732194306,4.628960915766174,3.7198138711154307,2.1940523222807116,0.0,4.261709381170972,1.8923811872495484,4.251616970894646]
[0.29677306220323846,2.658524850271114,3.4098293818558116,1.8179290670325896,0.0,3.3738532600936866,1.0593712866420917,2.6360025219546803]
[2.3741844976259077,5.723647618818986,3.306501218769272,0.0,0.0,2.9552925173001086,2.0281980188703295,2.721034861372573]
[0.29677306220323846,2.783631902048578,3.4098293818558116,1.4418058117844677,0.8156606685129459,3.5641081431816763,0.5040313529037872,1.785679127775751]
[0.0,4.284916523378148,2.0665632617307947,2.1940523222807116,1.4577765139380312,5.466656974061574,6.905531349963264,2.806067200790466]
[1.4838653110161923,3.628104501546461,3.823142034201971,0.0,0.0,3.24701667136836,0.6066485145728218,2.5509701825367874]
[0.8903191866097153,2.4395875096605515,2.5832040771634937,2.0059906946566506,0.7635972215865877,3.931934250485123,0.748501649821193,2.210840824865216]
[2.9677306220323847,3.596827738602095,0.0,0.0

In [14]:
df =scaled_features.map(lambda x: (x, )).toDF(["Scaled_data"])

In [17]:
df.show()

+--------------------+
|         Scaled_data|
+--------------------+
|[1.78063837321943...|
|[0.29677306220323...|
|[2.37418449762590...|
|[0.29677306220323...|
|[0.0,4.2849165233...|
|[1.48386531101619...|
|[0.89031918660971...|
|[2.96773062203238...|
|[0.59354612440647...|
|[2.37418449762590...|
|[1.18709224881295...|
|[2.96773062203238...|
|[2.96773062203238...|
|[0.29677306220323...|
|[1.48386531101619...|
|[2.07741143542266...|
|[0.0,3.6906580274...|
|[2.07741143542266...|
|[0.29677306220323...|
|[0.29677306220323...|
+--------------------+
only showing top 20 rows



In [18]:
sc.stop()